In [1]:
# coding=utf-8
from __future__ import print_function

import json
import os
import pickle
import sys
from itertools import chain

import numpy as np

from asdl.asdl import ASDLGrammar
from asdl.hypothesis import Hypothesis
from datasets.wikisql.lib.common import detokenize
from datasets.wikisql.lib.dbengine import DBEngine
from datasets.wikisql.lib.query import Query
from asdl.lang.sql.sql_transition_system import SqlTransitionSystem, sql_query_to_asdl_ast, asdl_ast_to_sql_query
from datasets.wikisql.utils import my_detokenize, find_sub_sequence
from asdl.transition_system import GenTokenAction
from components.action_info import ActionInfo
from components.vocab import VocabEntry, Vocab
from model.wikisql.dataset import WikiSqlExample, WikiSqlTable, TableColumn

In [2]:
from datasets.wikisql.dataset import load_dataset
from datasets.wikisql.dataset import get_action_infos

In [3]:
engine = DBEngine('tranx.0.2.0/data/wikisql/train.db')
data_file = 'dsl_parser/my_wikisql/wikisql_train.json'
grammar = ASDLGrammar.from_text(open('asdl/lang/sql/sql_asdl.txt').read())

transition_system = SqlTransitionSystem(grammar)

In [4]:
query = Query('1', 2)
query

SELECT MIN col1 FROM table

In [5]:
query.to_dict()

{'sel': '1', 'agg': 2, 'conds': []}

In [6]:
asdl_ast = sql_query_to_asdl_ast(query, grammar)
asdl_ast.sanity_check()
actions = transition_system.get_actions(asdl_ast)

In [23]:
actions.count

<function list.count(value, /)>

In [8]:
query_reconstr = asdl_ast_to_sql_query(asdl_ast)

In [9]:
query_reconstr

SELECT MIN col1 FROM table

In [35]:
q = {"sel": "Current series", "conds": [["Notes", 0, "New series began in June 2011"]], "agg": 0}

qu = Query.from_dict(q)

In [36]:
qu

SELECT  colCurrent series FROM table WHERE colNotes = New series began in June 2011

In [16]:
import pandas as pd
data = pd.read_csv('my_wikisql/train.csv')

In [18]:
data.head()

,question,sql
0,Tell me what the notes are for South Australia,SELECT Notes FROM table WHERE Current slogan =...
1,What is the current series where the new serie...,SELECT Current series FROM table WHERE Notes =...
2,What is the format for South Australia?,SELECT Format FROM table WHERE State/territory...
3,Name the background colour for the Australian ...,SELECT Text/background colour FROM table WHERE...
4,how many times is the fuel propulsion is cng?,SELECT COUNT Fleet Series (Quantity) FROM tabl...


In [27]:
qq = data.loc[1]['sql']
qq

'SELECT Current series FROM table WHERE Notes = New series began in June 2011'

In [ ]:
qq.